**You need to put the files in your drive and mount them**


In [ ]:
import pandas as pd
import numpy as np
import torch 
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.neural_network import MLPClassifier
data = pd.read_parquet('/content/drive/MyDrive/Class1.parquet', engine='pyarrow')
wdata = pd.read_parquet('/content/drive/MyDrive/Class2.parquet', engine='pyarrow')
validation = pd.read_parquet('/content/drive/MyDrive/validation.parquet', engine='pyarrow')
data =data.T
wdata = wdata.T
validation = validation.T
d = {'value': [0 for i in range(1677)]}
data_y =pd.DataFrame(data=d)
d2 = {'value': [1 for i in range(200)]}
wdata_y =pd.DataFrame(data=d2)
data = torch.tensor(np.array(data))
data_y = torch.tensor(np.array(data_y))
wdata =  torch.tensor(np.array(wdata))

In [ ]:
class Autoeconder(nn.Module):
    def __init__(self, input_size, bottleneck_size):
        super(Autoeconder, self).__init__()

        self.h1 = nn.Linear(input_size,int(2000))
        self.bottleneck = nn.Linear(2000,bottleneck_size)
        self.h3 = nn.Linear(bottleneck_size,2000)
        self.output = nn.Linear(2000,input_size)
        self.relu = nn.ReLU()
        self.batchnorm1 = nn.BatchNorm1d(2000)
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, input):
      h1 = self.h1(input)
      h1 = self.batchnorm1(h1)
      h1 = self.relu(h1)
      h1 = self.dropout(h1)
      bottleneck = self.bottleneck(h1)
      bottleneck = self.relu(bottleneck)
      h3= self.h3(bottleneck)
      h3 = self.batchnorm1(h3)
      h3 = self.relu(h3)
      h3 = self.dropout(h3)
      output = self.output(h3)
        
      return output

In [ ]:
def train_step_constructor(model,loss_f,optimizer):
  def train_step(batch_x, batch_y):
    p = model(batch_x)
    loss = loss_f(batch_x.view((-1,)), p.view((-1,)))
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return loss.item()
  return train_step

In [ ]:
from torch.utils.data import Dataset

class The_Dataset(Dataset):
    def __init__(self, x_arr, y_arr):
        self.x = torch.tensor(x_arr)
        self.y = torch.tensor(y_arr)
        
    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.x)

In [ ]:
train_data = The_Dataset(data, data_y)

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset=train_data, batch_size=100, shuffle=True)

In [ ]:
model = Autoeconder(61440,500)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)
mse_loss = nn.MSELoss(reduction='mean')
step = train_step_constructor(model,mse_loss,optimizer)
losses = []
val_losses = []

for epoch in range(100):
    model.train()  
    for batch_x,batch_y in train_loader:
      losses.append(step(batch_x.float(), batch_y.float()))
      print(losses[-1])
    if epoch == 10 :
      torch.save(model.state_dict(), "/content/sample_data/model")

In [ ]:
model = Autoeconder(61440,500)

model.load_state_dict(torch.load("/content/sample_data/model"))


In [ ]:
validation = torch.tensor(np.array(validation))
p = model(validation.float())
loss =mse_loss(validation.view((-1,)), p.view((-1,)))
loss

In [ ]:
validation = torch.tensor(np.array(validation))
losses=[]
results=[]
model.eval()

for i in validation:
  input = torch.unsqueeze(i, 0)
  print(input.size())
  p=model(input.float())
  loss =mse_loss(input.view((-1,)), p.view((-1,)))
  print(loss.item())
  losses.append(loss.item())
  if(loss.item()>0.55):
    results.append(1)
  else:
    results.append(0)
results

In [ ]:
df = pd.DataFrame({'instances' : ['instance_0','instance_1','instance_2','instance_3','instance_4','instance_5','instance_6','instance_7','instance_8','instance_9','instance_10','instance_11','instance_12','instance_13','instance_14','instance_15','instance_16','instance_17','instance_18','instance_19','instance_20','instance_21','instance_22','instance_23','instance_24','instance_25','instance_26','instance_27','instance_28','instance_29','instance_30','instance_31','instance_32','instance_33','instance_34','instance_35','instance_36','instance_37','instance_38','instance_39','instance_40','instance_41','instance_42','instance_43','instance_44','instance_45','instance_46','instance_47','instance_48','instance_49','instance_50','instance_51','instance_52','instance_53','instance_54','instance_55','instance_56','instance_57','instance_58','instance_59','instance_60','instance_61','instance_62','instance_63','instance_64','instance_65','instance_66','instance_67','instance_68','instance_69','instance_70','instance_71','instance_72','instance_73','instance_74','instance_75','instance_76','instance_77','instance_78','instance_79','instance_80','instance_81','instance_82','instance_83','instance_84','instance_85','instance_86','instance_87','instance_88','instance_89','instance_90','instance_91','instance_92','instance_93','instance_94','instance_95','instance_96','instance_97','instance_98','instance_99','instance_100','instance_101','instance_102','instance_103','instance_104','instance_105','instance_106','instance_107','instance_108','instance_109','instance_110','instance_111','instance_112','instance_113','instance_114','instance_115','instance_116','instance_117','instance_118','instance_119','instance_120','instance_121','instance_122','instance_123','instance_124','instance_125','instance_126','instance_127','instance_128','instance_129','instance_130','instance_131','instance_132','instance_133','instance_134','instance_135','instance_136','instance_137','instance_138','instance_139','instance_140','instance_141','instance_142','instance_143','instance_144','instance_145','instance_146','instance_147','instance_148','instance_149','instance_150','instance_151','instance_152','instance_153','instance_154','instance_155','instance_156','instance_157','instance_158','instance_159','instance_160','instance_161','instance_162','instance_163','instance_164','instance_165','instance_166','instance_167','instance_168','instance_169','instance_170','instance_171','instance_172','instance_173','instance_174','instance_175','instance_176','instance_177','instance_178','instance_179'],'label' : results})
df.rename(columns={0: "instances", 1: "label"})
df["label"] = df.label.astype(np.int)
df

In [ ]:
df.to_csv('/content/sample_data/test.csv',index=False)